In [80]:
import torch
import torch.nn as nn


In [81]:
class BeforeModules(nn.Module):
  def __init__(self,in_channels):
    super().__init__()
    self.in_channels = in_channels

    self.conv1 = nn.Conv2d(self.in_channels, 64, kernel_size  = 7, stride = 2,padding = 3 ,bias = False)
    self.bn1 = nn.BatchNorm2d( num_features = 64)
    self.relu = nn.ReLU()

    self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2 ,padding = 1)
    self.conv2 = nn.Conv2d(64, 64, kernel_size  = 1, stride = 1, bias = False)

    self.conv3 = nn.Conv2d(64, 192, kernel_size=3 ,stride = 1, padding = 1,bias = False)
    self.bn2 = nn.BatchNorm2d( num_features = 192)

    print("Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm")


  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)

    x = self.maxpool(x)
    x = self.conv2(x)
    x = self.relu(x)

    x = self.conv3(x)
    x = self.bn2(x)
    x = self.relu(x)

    return x

In [82]:
class Aux_Classifier(nn.Module):
  def __init__(self, in_channels, num_classes = 1000):
    super().__init__()

    self.in_channels = in_channels
    self.avgpool_aux = nn.AvgPool2d(kernel_size = 5, stride=3)
    self.conv_aux = nn.Conv2d(in_channels = self.in_channels, out_channels=128, kernel_size=1, stride=1, bias = False)
    self.bn = nn.BatchNorm2d(num_features=self.conv_aux.out_channels)
    self.relu = nn.ReLU()
    self.flatten = nn.Flatten()
    self.fc_aux = nn.Linear(in_features = 2048, out_features = 1024)#4*4*128
    self.bn_fc_aux = nn.BatchNorm1d(num_features=self.fc_aux.out_features)
    self.dropout_aux = nn.Dropout(p=0.7)
    self.classifier_aux = nn.Linear(in_features=self.fc_aux.out_features, out_features=num_classes)

  def forward(self, x):
    x = self.avgpool_aux(x)
    x = self.conv_aux(x)
    x = self.relu(x)
    x = self.bn(x)
    x = self.flatten(x)
    x = self.fc_aux(x)
    x = self.bn_fc_aux(x)
    x = self.relu(x)
    x = self.dropout_aux(x)
    out = self.classifier_aux(x)
    return out

In [83]:
class Inc_Module(nn.Module):
  def __init__(self, in_channels, out_channels_list):
    super().__init__()
    self.in_channels = in_channels
    self.relu = nn.ReLU()


    # self.conv1x1_l1--> conv layer of 1x1 size, in picture 1st layer from left
    self.conv1x1_l1 = nn.Conv2d( self.in_channels, out_channels_list[0] , stride = 1, kernel_size = 1, bias= False )
    self.convreduce3x3 = nn.Conv2d( self.in_channels, out_channels_list[1] , stride = 1, kernel_size = 1, bias= False )
    self.convreduce5x5 = nn.Conv2d( self.in_channels, out_channels_list[3] , stride = 1, kernel_size = 1,bias= False )
    self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 1,padding = 1)

    self.conv3x3 = nn.Conv2d(out_channels_list[1], out_channels_list[2],kernel_size = 3, padding = 1, stride = 1, bias = False)
    self.conv5x5 = nn.Conv2d(out_channels_list[3], out_channels_list[4],kernel_size = 5, padding = 2, stride = 1, bias = False)
    self.conv1x1_l4 = nn.Conv2d( self.in_channels, out_channels_list[5] , kernel_size = 1, stride = 1, bias= False )

    self.out_channels = sum(out_channels_list) - out_channels_list[1] - out_channels_list[3]

    print("Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm")



  def forward(self, x):
    # making 4 copies of x and applying respective operations in x1, x2, x3, x4

    x1 = self.conv1x1_l1(x)
    x1 = self.relu(x1)

    x2 = self.convreduce3x3(x)
    x2 = self.relu(x2)
    x2 = self.conv3x3(x2)
    x2 = self.relu(x2)

    x3 = self.convreduce5x5(x)
    x3 = self.relu(x3)
    x3 = self.conv5x5(x3)
    x3 = self.relu(x3)

    x4 = self.maxpool(x)
    x4 = self.conv1x1_l4(x4)

    out = torch.cat((x1,x2,x3,x4), dim = 1)


    return out



In [84]:
class Inception_imp(nn.Module):
  def __init__(self, in_channels = 3, num_classes = 1000):
    super().__init__()
    self.in_channels = in_channels
    self.num_classes = num_classes

    self.out_channels = {
                         '3a':[64,96,128,16,32,32],
                         '3b':[128,128,192,32,96,64],
                         '4a':[192,96,208,16,48,64],
                         '4b':[160,112,224,24,64,64],
                         '4c':[128,128,256,24,64,64],
                         '4d':[112,144,288,32,64,64],
                         '4e':[256,160,320,32,128,128],
                         '5a':[256,160,320,32,128,128],
                         '5b':[384,192,384,48,128,128]
                       }
    self.before_modules = BeforeModules(self.in_channels)

    self.module3 = self._build_module(192,{key:value for key,value in self.out_channels.items() if '3' in key})


    #self.module4 = self._build_module(self.module3[-1].out_channels,{key:value for key,value in self.out_channels.items() if '4' in key},aux_pos = true, num_classes = self.num_classes)
    self.Inc_block4a = Inc_Module(in_channels = self.module3[-1].out_channels, out_channels_list = self.out_channels['4a'])
    self.Inc_block4b = Inc_Module(in_channels = self.Inc_block4a.out_channels, out_channels_list =  self.out_channels['4b'])
    self.aux_classifier4b = Aux_Classifier(in_channels = self.Inc_block4a.out_channels, num_classes = self.num_classes)
    self.Inc_block4c = Inc_Module(in_channels = self.Inc_block4b.out_channels,  out_channels_list = self.out_channels['4c'])
    self.Inc_block4d = Inc_Module(in_channels = self.Inc_block4c.out_channels,  out_channels_list = self.out_channels['4d'])
    self.Inc_block4e = Inc_Module(in_channels = self.Inc_block4d.out_channels,  out_channels_list = self.out_channels['4e'])
    self.aux_classifier4e = Aux_Classifier(in_channels = self.Inc_block4d.out_channels, num_classes = self.num_classes)


    self.module5= self._build_module(self.Inc_block4e.out_channels,{key:value for key,value in self.out_channels.items() if '5' in key})
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2,padding = 1)

    self.avg_pool = nn.AvgPool2d(kernel_size = 7, stride = 1)
    self.dropout = nn.Dropout(0.4)
    self.fc = nn.Linear(in_features = self.module5[-1].out_channels, out_features = self.num_classes)


  def _build_module(self, in_channels,module_dict):
    module = []
    out_channels_list = list(module_dict.values())
    for i in range(len(module_dict)):

      block = Inc_Module(in_channels, out_channels_list[i])
      module.append(block)
      in_channels = block.out_channels

    return nn.Sequential(*module)

  def forward(self,x):
    aux_out1 = None
    aux_out2 = None

    x = self.before_modules(x)
    x = self.module3(self.maxpool(x))


    x = self.Inc_block4a(self.maxpool(x))
    if self.training:
      aux_out1 = self.aux_classifier4b(x)
    x = self.Inc_block4b(x)
    x = self.Inc_block4c(x)
    x = self.Inc_block4d(x)
    if self.training:
      aux_out2 = self.aux_classifier4e(x)
    x = self.Inc_block4e(x)


    x = self.module5(self.maxpool(x))

    x = self.avg_pool(x)
    x = self.dropout(x)
    x = torch.flatten(x,1)
    x = self.fc(x)

    return x, aux_out1, aux_out2



In [85]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim


transform = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])

train_dataset = torchvision.datasets.FashionMNIST(root="data",
                                                         train=True,
                                                         download=True,
                                                         transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root="data",
                                                 train=False,
                                                 download=True,
                                                 transform=transform)

In [86]:
batchsize = 64
train_dataloader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

In [87]:
model = Inception_imp(in_channels=1, num_classes=10)

Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
Sairammmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm


In [88]:
optimizer = optim.SGD(params=model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss(reduction='sum')

In [89]:
import torch.nn.functional as F
from tqdm import tqdm

def train_epoch(model, criterion, optimizer, train_loader, test_loader, epoch, device):
  loss_train, loss_val = 0., 0.
  acc_train, acc_val = 0., 0.
  ntrain = len(train_loader.dataset)
  nval = len(test_loader.dataset)

  with tqdm(train_loader, unit='batches') as pbar:
    for i, (images, labels) in enumerate(pbar):
      pbar.set_description(f"Epoch {epoch}")

      images, labels = images.to(device), labels.to(device).to(torch.long).view(-1)

      # training and back-propagation
      optimizer.zero_grad()
      out_main, out_aux_1, out_aux_2 = model(images)

      main_loss = criterion(out_main, labels)
      aux_1_loss = criterion(out_aux_1, labels)
      aux_2_loss = criterion(out_aux_2, labels)
      loss = main_loss + 0.3*aux_1_loss + 0.3*aux_2_loss

      loss.backward()
      optimizer.step()

      # calculating accuracy
      with torch.no_grad():
        probs = F.softmax(out_main, dim=1).cpu()
        preds = torch.max(probs, dim=1)[1]
        acc_train += (preds == labels.cpu()).sum()
        loss_train += loss.item()

      # validate
      if i%100 == 0 or i == len(train_loader) - 1:
        loss_val, acc_val = validate(model, criterion, test_loader, device)

      if i < len(train_loader) - 1:
        denom = images.shape[0] * (i+1)
      else:
        denom = ntrain

      pbar.set_postfix(train_loss=loss_train/denom, valid_loss=loss_val/nval, train_acc=acc_train/denom, valid_acc=acc_val/nval)

  return loss_train/ntrain, loss_val/nval, acc_train/ntrain, acc_val/nval

def validate(model, criterion, val_loader, device):
  acc_val = 0.
  loss_val = 0.
  with torch.no_grad():
    for val_images, val_labels in val_loader:
      val_images = val_images.to(device)
      val_labels = val_labels.to(device).to(torch.long).view(-1)

      val_output, _, _ = model(val_images)
      val_loss = criterion(val_output, val_labels)
      loss_val += val_loss.item()

      probs = F.softmax(val_output, dim=1).cpu()
      preds = torch.max(probs, dim=1)[1]
      acc_val += (preds == val_labels.cpu()).sum()

  return loss_val, acc_val

def train(model, criterion, optimizer, train_loader, val_loader, epochs, device):
  train_losses = []
  val_losses = []
  train_accs = []
  val_accs = []

  for epoch in range(epochs):
    train_loss, val_loss, train_acc, val_acc = train_epoch(model, criterion, optimizer, train_loader, val_loader, epoch, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

  return train_losses, val_losses, train_accs, val_accs

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
criterion = criterion.to(device)
epochs = 2

train_losses, val_losses, train_accs, val_accs = train(model, criterion, optimizer, train_dataloader, test_dataloader, epochs, device)

Epoch 0:   0%|          | 0/938 [00:00<?, ?batches/s]

In [ ]:
import torchvision
from torchsummary import summary
inc  = torchvision.models.googlenet()
summary(inc, (3, 224, 224))

In [ ]:
inc